# Import Section
---

In [1]:
"""
This script help user to training VWW model, converting to int8 quantization TFLite, testing and deployment. 
"""
import shutil
import os

import ipywidgets as widgets
from ipywidgets import interact_manual
from ipywidgets import Layout, Box, Dropdown, Label
from IPython.display import display, HTML
from IPython.display import clear_output
from ipyfilechooser import FileChooser

# Widgets Control Section
---

In [2]:
class TrainConfigAndCmdsWidgets:
    """
    A class to create and manage widgets for configuring and running training, testing, and deployment of a machine learning model.
    """

    # ------------------#
    # init all the
    # ------------------#
    def __init__(self):

        self.tflite_file_loc = ""

        form_item_layout = Layout(
            display="flex",
            flex_flow="row",
            justify_content="space-between",
        )

        # data prepare
        self.a_dp = widgets.Textarea(
            value=r"C:\Users\USER\image_classification\ML_tf2_object_detection_nu\image_dataset\coco-2017\train\data", placeholder="Type something", disabled=False, layout={"height": "40pt"}
        )
        self.b_dp = widgets.Textarea(
            value=r"C:\Users\USER\image_classification\ML_tf2_object_detection_nu\image_dataset\coco-2017\validation\data", placeholder="Type something", disabled=False, layout={"height": "40pt"}
        )
        self.c_dp = widgets.Textarea(
            value=r"C:\Users\USER\image_classification\ML_tf2_object_detection_nu\image_dataset\coco-2017\raw\instances_train2017.json",
            placeholder="Type something",
            disabled=False,
            layout={"height": "40pt"},
        )
        self.d_dp = widgets.Textarea(
            value=r"C:\Users\USER\image_classification\ML_tf2_object_detection_nu\image_dataset\coco-2017\raw\instances_val2017.json",
            placeholder="Type something",
            disabled=False,
            layout={"height": "40pt"},
        )
        self.e_dp = widgets.Text(value=r"dataset\datasets\coco_person", placeholder="Type something", disabled=False)
        self.f_dp = widgets.Text(value="person", placeholder="Type something", disabled=False)
        self.g_dp = widgets.Button(description="Run", layout=Layout(width="30%", height="30px"), button_style="success")

        form_data_prepare_items = [
            Box([Label(value="Train Image Dir"), self.a_dp], layout=form_item_layout),
            Box([Label(value="Val Image Dir"), self.b_dp], layout=form_item_layout),
            Box([Label(value="Train Annotations File"), self.c_dp], layout=form_item_layout),
            Box([Label(value="Val Annotations File"), self.d_dp], layout=form_item_layout),
            Box([Label(value="Output Dir"), self.e_dp], layout=form_item_layout),
            Box([Label(value="Object Name"), self.f_dp], layout=form_item_layout),
            Box([Label(value="Start to Execute"), self.g_dp], layout=form_item_layout),
        ]

        self.form_data_prepare_cmd = Box(
            form_data_prepare_items,
            layout=Layout(
                display="flex",
                flex_flow="column",
                border="solid 1px lightblue",
                align_items="stretch",
                width="100%",
            ),
        )

        # train
        self.a_ta = widgets.Text(value="vww_v2_96_035", placeholder="Type something", disabled=False)
        self.e_ta = Dropdown(options=["mobilenetv2", "mobilenetv1", "mobilenetv3"])
        self.f_ta = Dropdown(options=["rgb", "grayscale"])
        self.b_ta = widgets.IntSlider(value=256, min=4, max=512, step=4)
        self.c_ta = widgets.IntSlider(value=96, min=64, max=192, step=32)
        self.d_ta = widgets.FloatSlider(value=0.35, min=0.1, max=1.5, step=0.05)

        self.h_ta = widgets.BoundedIntText(value=5, min=1, max=500, step=1)
        self.i_ta = widgets.Text(value="0.0001", placeholder="Type something", disabled=False)

        self.j_ta = Dropdown(value="1: Train model & Convert to tflite", options=["0: Show the data pictures only", "1: Train model & Convert to tflite", "2: Convert to tflite"])
        self.l_ta = widgets.Button(description="Run", layout=Layout(width="30%", height="30px"), button_style="success")

        self.k_ta = widgets.BoundedIntText(value=50, min=0, max=300, step=1, disabled=False)
        self.fine_tune_la = Label(value="Freezing Layers of Fine-Tuning")
        self.k_ta_box = Box([self.fine_tune_la, self.k_ta], layout=form_item_layout)

        self.m_ta = widgets.BoundedIntText(value=50, min=1, max=1000, step=1)
        self.fine_tune_lb = Label(value="Epochs of Fine-Tuning")
        self.m_ta_box = Box([self.fine_tune_lb, self.m_ta], layout=form_item_layout)

        self.n_ta = widgets.Text(value="0.00001", placeholder="Type something", disabled=False)
        self.fine_tune_lc = Label(value="Learning Rate of Fine-Tuning")
        self.n_ta_box = Box([self.fine_tune_lc, self.n_ta], layout=form_item_layout)

        form_train_items = [
            Box([Label(value="Project Name"), self.a_ta], layout=form_item_layout),
            Box([Label(value="Model Type"), self.e_ta], layout=form_item_layout),
            Box([Label(value="Color Mode"), self.f_ta], layout=form_item_layout),
            Box([Label(value="Batch Size"), self.b_ta], layout=form_item_layout),
            Box([Label(value="Image Size"), self.c_ta], layout=form_item_layout),
            Box([Label(value="Alpha"), self.d_ta], layout=form_item_layout),
            Box([Label(value="Epochs"), self.h_ta], layout=form_item_layout),
            Box([Label(value="Learning Rate"), self.i_ta], layout=form_item_layout),
            Box([Label(value="Switch Mode"), self.j_ta], layout=form_item_layout),
            self.k_ta_box,
            self.m_ta_box,
            self.n_ta_box,
            Box([Label(value="Start to Execute"), self.l_ta], layout=form_item_layout),
        ]

        self.form_output_train_cmd = Box(
            form_train_items,
            layout=Layout(
                display="flex",
                flex_flow="column",
                border="solid 1px lightblue",
                align_items="stretch",
                width="100%",
            ),
        )

        # test
        self.a_tt = widgets.Button(description="Setting", layout=Layout(width="30%", height="30px"), button_style="success")
        self.c_tt = widgets.Button(description="Run", layout=Layout(width="30%", height="30px"), button_style="success")

        form_test_items = [
            Box([Label(value="The tflite test setting is derivative from train setting.")], layout=form_item_layout),
            Box([Label(value="Choose the tflite file"), self.a_tt], layout=form_item_layout),
            Box([Label(value="Start to Test"), self.c_tt], layout=form_item_layout),
        ]

        self.form_output_test_cmd = Box(
            form_test_items,
            layout=Layout(
                display="flex",
                flex_flow="column",
                border="solid 3px lightgreen",
                align_items="stretch",
                width="50%",
            ),
        )

        # convert model cpp
        self.a_cm = widgets.Text(value=r"..\workspace\\vww_v2_96_035\\tflite_model", placeholder="Type something", disabled=False)
        self.b_cm = widgets.Text(value="vww_mobilenetv2_0.35_96_96_int8quant.tflite", placeholder="Type something", disabled=False)
        self.c_cm = widgets.Text(value=r"..\workspace\\vww_v2_96_035\\tflite_model\\c", placeholder="Type something", disabled=False)
        self.e_cm = widgets.Button(description="Setting", layout=Layout(width="30%", height="30px"), button_style="success")
        self.d_cm = widgets.Button(description="Run", layout=Layout(width="30%", height="30px"), button_style="success")
        self.g_cm = widgets.Checkbox(value=False, disabled=False, indent=False)

        form_convert_items_paths = [
            Box([Label(value="Vela Compiler"), self.g_cm], layout=form_item_layout),
            Box([Label(value="Choose the tflite file"), self.e_cm], layout=form_item_layout),
            Box([Label(value="Model Src Dir"), self.a_cm], layout=form_item_layout),
            Box([Label(value="Model Src File"), self.b_cm], layout=form_item_layout),
            Box([Label(value="Gen Src Dir"), self.c_cm], layout=form_item_layout),
        ]

        form_convert_items = [
            Box(form_convert_items_paths, layout=Layout(display="flex", flex_flow="column", justify_content="center", border="dotted 3px lightblue", align_items="stretch", width="70%")),
            Box([Label(value="Convert to cpp & Vela"), self.d_cm], layout=form_item_layout),
        ]

        self.form_output_convert_cmd = Box(
            form_convert_items,
            layout=Layout(
                display="flex",
                flex_flow="column",
                justify_content="center",
                border="solid 3px lightgreen",
                align_items="stretch",
                width="70%",
            ),
        )

    # ------------------#
    # help functions section
    # ------------------#
    def move_allfiles(self, src_folder, dst_folder):
        """
        Moves all files and directories from the source folder to the destination folder.
        Example:
            move_allfiles('/path/to/source', '/path/to/destination')
        """
        files = os.listdir(src_folder)
        for f in files:
            fullpath = os.path.join(src_folder, f)
            if os.path.isdir(fullpath):  # copy whole folder
                shutil.move(fullpath, dst_folder)
                print(f"Copy finish: {f}")

    def show_headline(self, output):
        """
        Displays a headline with the specified output text in a styled HTML format.
        Args:
            output (str): The text to be displayed as the headline.
        """
        html0 = widgets.HTML(value=f"<b><font color='lightblue'><font size=4>{output}</b>")
        display(html0)

    # ------------------#
    # Show main widgets which combine all widgets.
    # ------------------#
    def show_main(self):
        """
        Displays the main interface for data preparation and training settings.
        This method creates and displays a user interface using ipywidgets, including an accordion for data preparation and training configuration, 
        and a tab for training, testing, and deployment. It also sets up interactive output and button click handlers for various actions.
        """

        intro_text = "Please Choose the setting of data prepare & train"
        html_widget = widgets.HTML(value=f"<b><font color='lightgreen'><font size=6>{intro_text}</b>")
        display(html_widget)

        # Create an accordion and put the 2 boxes
        accordion = widgets.Accordion(
            children=[
                self.form_data_prepare_cmd,
                self.form_output_train_cmd,
            ]
        ).add_class("parentstyle")
        display(HTML("<style>.parentstyle > .p-Accordion-child > .p-Collapse-header{background-color:green}</style>"))
        accordion.set_title(0, "COCO Dataset Prepare")
        accordion.set_title(1, "Configure the Training")

        # Create a box combining with 2 elements
        box_data_train = Box(
            [accordion],
            layout=Layout(
                display="flex",
                flex_flow="column",
                border="solid 3px lightgreen",
                align_items="stretch",
                width="50%",
            ),
        )

        # Create a tab and put the 2 boxes
        tab = widgets.Tab(children=[box_data_train, self.form_output_test_cmd, self.form_output_convert_cmd]).add_class("parentstyle")
        tab_contents = ["Train", "Test", "Deployment"]
        tab.titles = tab_contents

        output_widgets = widgets.Output(layout=Layout(border="1px solid green"))

        def act_para(*, a_dp, b_dp, c_dp, d_dp, e_dp, f_dp, a_ta, b_ta, c_ta, d_ta, e_ta, f_ta, h_ta, i_ta, j_ta, k_ta, m_ta, n_ta, a_cm, b_cm, c_cm, g_cm):

            # If any value is changed, clear the widgets
            with output_widgets:
                output_widgets.clear_output()

            if f_ta.count("rgb"):
                self.k_ta.layout.visibility = "visible"
                self.fine_tune_la.layout.visibility = "visible"
                self.m_ta.layout.visibility = "visible"
                self.fine_tune_lb.layout.visibility = "visible"
                self.n_ta.layout.visibility = "visible"
                self.fine_tune_lc.layout.visibility = "visible"
            else:
                self.k_ta.layout.visibility = "hidden"
                self.fine_tune_la.layout.visibility = "hidden"
                self.m_ta.layout.visibility = "hidden"
                self.fine_tune_lb.layout.visibility = "hidden"
                self.n_ta.layout.visibility = "hidden"
                self.fine_tune_lc.layout.visibility = "hidden"

        # ------------------#
        # widgets.Accordion's interactive input with action function `act_para()`
        # ------------------#
        out_inter = widgets.interactive_output(
            act_para,
            {
                "a_dp": self.a_dp,
                "b_dp": self.b_dp,
                "c_dp": self.c_dp,
                "d_dp": self.d_dp,
                "e_dp": self.e_dp,
                "f_dp": self.f_dp,
                "a_ta": self.a_ta,
                "b_ta": self.b_ta,
                "c_ta": self.c_ta,
                "d_ta": self.d_ta,
                "e_ta": self.e_ta,
                "f_ta": self.f_ta,
                "h_ta": self.h_ta,
                "i_ta": self.i_ta,
                "j_ta": self.j_ta,
                "k_ta": self.k_ta,
                "m_ta": self.m_ta,
                "n_ta": self.n_ta,
                "a_cm": self.a_cm,
                "b_cm": self.b_cm,
                "c_cm": self.c_cm,
                "g_cm": self.g_cm,
            },
        )
        display(tab, out_inter)
        display(output_widgets)

        # ------------------#
        # buttoms interactive sections
        # ------------------#

        def on_button_clicked_dataprepare(b):
            with output_widgets:
                clear_output()
                print("Coco VWW dataprepare. . .")
                self.run_coco_dataprepare()

        self.g_dp.on_click(on_button_clicked_dataprepare)

        def on_button_clicked_train(b):
            with output_widgets:
                clear_output()
                print("Train. . .")
                self.run_train()

        self.l_ta.on_click(on_button_clicked_train)

        def on_button_clicked_choose_tflite(b):
            with output_widgets:
                clear_output()
                self.choose_tflite()

        self.e_cm.on_click(on_button_clicked_choose_tflite)

        def on_button_clicked_cpp(b):
            with output_widgets:
                clear_output()
                print("Convert to cpp & Vela. . .")
                self.convert_tflu()

        self.d_cm.on_click(on_button_clicked_cpp)

        def on_button_clicked_choose_tflite_test(b):
            with output_widgets:
                clear_output()
                self.choose_tflite()

        self.a_tt.on_click(on_button_clicked_choose_tflite_test)

        def on_button_clicked_test(b):
            with output_widgets:
                clear_output()
                self.run_test_tflite()

        self.c_tt.on_click(on_button_clicked_test)

    # ------------------#
    # execute functions sections, handle all the interactive events
    # ------------------#
    def run_coco_dataprepare(self):
        """
        Prepares COCO dataset for training and validation.
        This method is a placeholder for running the COCO dataset preparation script.
        It prints a test message and contains commented-out code for executing the 
        `create_coco_vww_tf_record.py` script with the necessary arguments.
        """
        %run create_coco_vww_tf_record.py --train_image_dir $self.a_dp.value --val_image_dir $self.b_dp.value \
        --train_annotations_file $self.c_dp.value --val_annotations_file $self.d_dp.value \
        --object_name $self.f_dp.value --output_dir $self.e_dp.value

    def choose_tflite(self):
        """
        Prompts the user to choose a TensorFlow Lite (.tflite) file for conversion.
        This method uses a file chooser widget to allow the user to select a .tflite file
        from the current working directory's "workspace" folder. The selected file's path
        and name are then processed and displayed, and relevant attributes are updated
        based on the user's selection.
        """

        path_ftflite = os.path.join(os.getcwd(), "workspace")
        f_tflite = FileChooser(path_ftflite)
        # Restrict navigation to /Users
        f_tflite.sandbox_path = os.getcwd()
        f_tflite.filter_pattern = ["*.tflite"]
        f_tflite.title = "<b><font color='lightblue'><font size=4>Choose the Tflite for Converting.</b>"
        display(f_tflite)

        def act_test():
            self.tflite_file_loc = f_tflite.selected
            work_dir_name = os.getcwd().split("\\")[-1]
            m_src_dir = r".." + f_tflite.selected_path.split(work_dir_name)[-1]
            m_src_tflite = f_tflite.selected.split("\\")[-1]
            print(f"The chosen dir: {m_src_dir}")
            print(f"The chosen tflite: {m_src_tflite}")
            self.a_cm.value = m_src_dir
            self.b_cm.value = m_src_tflite
            if self.g_cm.value:
                self.c_cm.value = os.path.join(m_src_dir, "vela")
            else:
                self.c_cm.value = os.path.join(m_src_dir, "c")

            print("Finish!")

        evt = interact_manual(act_test)
        evt.widget.children[0].description = "Set this file"  # because there are 3 parameter of the evt
        evt.widget.children[0].button_style = "primary"

    def run_train(self):
        """
        Executes the training process for the Visual Wake Words (VWW) model.
        This method constructs a command to run the `train_vww.py` script with various parameters
        extracted from the instance attributes. It also saves the constructed command to a file
        for record-keeping purposes.
        Saves:
            A text file named `train_cmd_record.txt` in the `workspace/<project_name>` directory,
        """
        sw_num = int(self.j_ta.value.split(":")[0])
        dataset_name = self.e_dp.value.split("\\")[-1]

        %run train_vww.py --proj_name $self.a_ta.value --dataset $dataset_name \
        --model-prefix $self.e_ta.value --COLOR_MODE $self.f_ta.value --batch-size $self.b_ta.value \
        --input-height $self.c_ta.value --input-width $self.c_ta.value --alpha $self.d_ta.value \
        --epochs $self.h_ta.value --learning-rate $self.i_ta.value --switch_mode $sw_num\
        --fine_tune_at $self.k_ta.value  --epochs_fine $self.m_ta.value --learning-rate_fine $self.n_ta.value

        # Save the cmd's record.
        path_cmd_record = os.path.join(os.getcwd(), "workspace", self.a_ta.value, "train_cmd_record.txt")
        with open(path_cmd_record, "w", encoding="utf-8") as file1:
            open_dir = f"cd {os.getcwd()} \n"
            cmd = f"python train_vww.py --proj_name {self.a_ta.value} --dataset {dataset_name} --model-prefix {self.e_ta.value} \
--COLOR_MODE {self.f_ta.value} --batch-size {self.b_ta.value} \
--input-height {self.c_ta.value} --input-width {self.c_ta.value} --alpha {self.d_ta.value} --epochs {self.h_ta.value} \
--learning-rate {self.i_ta.value} --switch_mode {sw_num} \
--fine_tune_at {self.k_ta.value} --epochs_fine {self.m_ta.value} --learning-rate_fine {self.n_ta.value}"
            file1.writelines([open_dir, cmd])

        print("Finish Train!!")

    def run_test_tflite(self):
        """
        Runs a test using a TensorFlow Lite model on a specified validation dataset.
        This method prints the location of the TensorFlow Lite file and the validation dataset being used.
        It also contains a commented-out command for running a training script with various parameters.
        """
        dataset_name = self.e_dp.value.split("\\")[-1]
        print(f"The tflite file: {self.tflite_file_loc} The testing validation dataset: {self.e_dp.value}")

        %run train_vww.py --proj_name $self.a_ta.value --dataset $dataset_name \
        --model-prefix $self.e_ta.value --COLOR_MODE $self.f_ta.value --batch-size $self.b_ta.value \
        --input-height $self.c_ta.value --input-width $self.c_ta.value --alpha $self.d_ta.value \
        --epochs $self.h_ta.value --learning-rate $self.i_ta.value --switch_mode $3\
        --fine_tune_at $self.k_ta.value  --epochs_fine $self.m_ta.value --learning-rate_fine $self.n_ta.value\
        --TFLITE_F $self.tflite_file_loc

        print("Finish !!")

    def convert_tflu(self):
        """
        Converts the current model to TensorFlow Lite format.
        """

        print("Create")
        %run exebat.py --SRC_DIR $self.a_cm.value --SRC_FILE $self.b_cm.value --GEN_DIR $self.c_cm.value --VELA_EN $self.g_cm.value

# Run Section
---
- The detail description of all the parameters and each step meaning is here [meaning](#id-train_evl_monitor)
- In this notebook step, you have alreay finish the dataset prepared. If not, please go to `image_dataset\create_data.ipynb`.

In [4]:
act = TrainConfigAndCmdsWidgets()
act.show_main()

HTML(value="<b><font color='lightgreen'><font size=6>Please Choose the setting of data prepare & train</b>")

Output()

Output(layout=Layout(border_bottom='1px solid green', border_left='1px solid green', border_right='1px solid g…